<a href="https://colab.research.google.com/github/maarongross/apriltag-imgs/blob/master/AprilTags.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pip._internal import main as pipmain
pipmain(['install', 'apriltag'])
pipmain(['install', 'dxfgrabber'])

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import apriltag
import dxfgrabber
import numpy as np
import time

April Tag Detector Demonstration

In [ ]:
#Reading in the file, converting it to a black and white img, and resizing
img = cv2.imread('/content/drive/MyDrive/(1) UCSD/(6) Fall 2022/MAE-148/AprilTags/Track+Tags.PNG')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img,(4208,3120), interpolation = cv2.INTER_AREA)

apriltag.DetectorOptions() # Setting up the detector options object
options = apriltag.DetectorOptions(families="tag16h5",quad_decimate=1)
detector = apriltag.Detector(options) # Instantiating the detector, with the detector options

result = detector.detect(img) #Getting the resutls from the detector, with the image input

img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB) # Converting the origonal image back to color, so that we can draw bounding boxes and stuff
#loop to draw bounding boxes, and mark centers
for r in result:

  (A,B,C,D) = r.corners 
  corner_B = (int(B[0]), int(B[1]))
  corner_C = (int(C[0]), int(C[1]))
  corner_D = (int(D[0]), int(D[1]))
  corner_A = (int(A[0]), int(A[1]))
  
  cv2.line(img, corner_A, corner_B, (0,255,0), 5)
  cv2.line(img, corner_B, corner_C, (0,255,0), 5)
  cv2.line(img, corner_C, corner_D, (0,255,0), 5)
  cv2.line(img, corner_D, corner_A, (0,255,0), 5)

  (cX,cY) = (int(r.center[0]), int(r.center[1]))
  cv2.circle(img,(cX,cY), 5, (0, 0, 255), -1)

  tagFamily = r.tag_family.decode("utf-8")
  tagID = str(r.tag_id)
  print(tagID)
  cv2.putText(img, "tagID: " + tagID, (corner_A[0]+50, corner_A[1] - 30),
              cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 10)
  
  print("[INFO] tag ID: {}".format(tagID))

cv2_imshow(img) # Displaying the final image

In [ ]:
#Simple dict to map tags to values
try:
  checkPointInfo = {10 : "Velocity = 10 \n Checkpoint #1",
                    20 : "Velocity = 50 \n Checkpoint #2",
                    30 : "Velocity = 10 \n Checkpoint #3"}
  print(checkPointInfo[int(tagID)])
except:
  print('No mapped check-point detected')



No mapped check-point detected


Machinery to Run Motion Allong a Polyline




In [ ]:
# Loading track map and resizing
track = cv2.imread('/content/drive/MyDrive/(1) UCSD/(6) Fall 2022/MAE-148/AprilTags/track backround.png')
track = cv2.resize(track,(1920,1080), interpolation = cv2.INTER_AREA)

# Image Zero-Zero
(zX, zY) = (int(1920/2), int(1080/2))
pic_center = (zX + 20, zY - 40)

# Definition of the polyline class, made up from objects of the obj class
class polyLine():
  objects = None
  object_index = 0
  track_image = None
  image_scaler = None
  def __init__(self, points, track_image, image_scaler):
    self.track_image = track_image
    self.image_scaler = image_scaler
    self.assembleFromObjects(points)
  def assembleFromObjects(self, cps):
    #declaring a list of objs, so its easy to iterate through and instantiate the objects
    objs = list()
    i = 0
    while i < len(cps)-1:
      start = (self.trans_dist2img(cps[i][0]), self.trans_dist2img(cps[i][1]))
      end = (self.trans_dist2img(cps[i+1][0]), self.trans_dist2img(cps[i+1][1]))
      
      objs.append(obj(cps[i][2], 
                      pic_center, 
                      start, 
                      end, 
                      cps[i][3], 
                      (self.trans_dist2img(cps[i][4][0]), self.trans_dist2img(cps[i][4][1])),
                      cps[i][5],
                      cps[i][6],
                      self.track_image, 
                      self.image_scaler))
      i +=1

    # Linking objects between themselves inside the object class
    # Forward
    i = 0
    while i < len(objs)-1:
      objs[i].nextObj = objs[i+1]
      i+=1
    objs[len(objs)-1].nextObj = objs[0]
    # Backwards
    i = len(objs)-1
    while i-1 > 0:
      objs[i].prevObj = objs[i-1]
      i-=1
    objs[0].prevObj = objs[len(objs)-1]
      
    #Store the list of objects
    self.objects = objs 
    print('Number of objects read in', len(self.objects))
  def getMotion(self, velocity, time):
      runoff = self.objects[self.object_index].getMotion(velocity, time)
      self.track_image = self.objects[self.object_index].track_image
      if runoff == 'a':
        pass
      if runoff != None:
        if self.object_index+1 >= len(self.objects):
          self.object_index = 0
        else:
          self.object_index +=1
  def getPolyLength(self):
    sum = 0
    for p in self.objects:
      sum += self.trans_img2dist_float(p.linearExtent)
    return sum
  def trans_dist2img(self, x):
    return(int(x*self.image_scaler))
  def trans_dist2img_float(self, x):
    return(float(x*self.image_scaler))
  def trans_img2dist(self, x):
    scaler = (1/self.image_scaler)
    return(int(x*scaler))
  def trans_img2dist_float(self, x):
    scaler = 1/self.image_scaler
    return(float(x*scaler))
 

# Definition of the object class, this stores individual segements of a polyline
class obj():
  track_image = None
  image_scaler = None
  
  objType = None
  pic_center = (-1,-1) #center of the picture where this obj is located  
  start = (-1,-1) # coordinates in meters from center of the track
  end = (-1,-1) # coordinates in meters from the center of the track
  radius_point = (-1,-1) # if the obj is a circle, location of the radius point
  angle = -1 # angle in radians measured from the horizontal
  distance = -1 # linear distance between begining and end point
  radius = -1 # if the object is a circle, it will need a radius value
  arc_orientation = -1
  color = (-1,-1,-1) # color to display the object as (b, g, r)
  
  theta = -1 
  theta_start = -1

  linearPos = 0 # Linear position allong the object
  linearExtent = 0 # Total linear distance of the object

  prevObj = None # To carry over motion to the next object
  nextObj = None # To carry over motion to the previous object

  def __init__(self, object_type, pic_center, start, end, color, radius_point, arc_orientation, approach_direction, track_image, image_scaler):
    self.track_image = track_image
    self.image_scaler = image_scaler
    self.objType = object_type
    self.pic_center = (pic_center[0], pic_center[1])
    self.start = (start[0],start[1])
    self.realPos = (start[0], start[1])
    self.end = (end[0], end[1])
    self.color = color
    self.radius_point = (radius_point[0], radius_point[1])
    self.arc_orientation = arc_orientation
    self.approach_direction = approach_direction

    #Setting Specific values for lines
    if self.objType == 'line':
      self.getAngle()
      self.getDistance()
    #Setting Specific values for circles
    if self.objType == 'circle':
      self.getRadius()
      self.get_start_angle()
      self.get_arc_length()
  
  # Transform methods are either scaler transforms, or simple shifts in space
  def trans_obj_2_img(self,x,y): # transforms object-space measured in pixels to image-space
    return((int(self.pic_center[0]+x), int(self.pic_center[1]+y)))
  def trans_img_2_obj(self,x,y): # transforms image-space to object-space measured in pixels
    return((int(x-self.pic_center[0]), int(y-self.pic_center[1])))
  def trans_scaler_obj_2_img(self,x): # 
    return((int(self.pic_center[0]+x)))
  def trans_scaler_img_2_obj(self,x):
    return((int(x-self.pic_center[0])))
  def trans_dist2img(self, x):
    return(int(x*self.image_scaler))
  def trans_dist2img_float(self, x):
    return(float(x*self.image_scaler))
  def trans_img2dist(self, x):
    scaler = (1/self.image_scaler)
    return(int(x*scaler))
  def trans_img2dist_float(self, x):
    scaler = 1/self.image_scaler
    return(float(x*scaler))
  
  # Get conditions inheirent to the definition of each object
  def getAngle(self):
    # Vector 'a' is defined by the start and end points where its normal is calculated and its angle coincident with the x-axis
    a = (self.end[0]-self.start[0], self.end[1]-self.start[1])
    anorm = (a[0]/float(np.sqrt(abs(float(np.square(a[0]))+float(np.square(a[1]))))), 
             a[1]/float(np.sqrt(abs(float(np.square(a[0]))+float(np.square(a[1]))))))
    self.angle = float(np.arccos(anorm[0]))

    # Correcting for vectors with end point below the horizontal
    if anorm[1] < 0:
      self.angle = -self.angle
    else:
      pass  
  def getDistance(self): # gets distance in meters
    a = (self.end[0]-self.start[0],self.end[1]-self.start[1])
    self.linearExtent = np.sqrt(np.square(a[0])+np.square(a[1]))
  def getRadius(self): # gets radius value in pixles
    #Calculares absolute value of the vector made be the radius and start points in image space
    xDiff = np.square(self.radius_point[0]-self.start[0])
    yDiff = np.square(self.radius_point[1]-self.start[1])
    self.radius = int(np.sqrt(xDiff + yDiff))
  def get_start_angle(self): # Gets angle from the horizontal to clock motion allong the circle    
    if self.objType == 'circle':
      #gets angle between the start point and the horizontal measured in object space
      (local_x, local_y) = (self.start[0]-self.radius_point[0], self.start[1]-self.radius_point[1])
      local_theta = np.arcsin(local_y/self.radius)
      #if the measured value is in the left half plane, correct for angle greater than 90
      if local_x < 0:
        local_theta = np.pi + local_theta 

      self.theta = local_theta
      self.theta_start = local_theta
    else:
      # This is not currnetly applicable for other object types
      pass
  def get_arc_length(self): # Gets arc length, so that the circular space can be transformed into linear space
    (aX,aY) = (self.start[0] - self.radius_point[0] , self.start[1] - self.radius_point[1])
    (bX,bY) = (self.end[0] - self.radius_point[0], self.end[1] - self.radius_point[1])
    aMag = np.sqrt(abs(np.square(aX)+np.square(aY)))
    bMag = np.sqrt(abs(np.square(bX)+np.square(bY)))
    (aX_norm, aY_norm) = (aX/aMag, aY/aMag) 
    (bX_norm, bY_norm) = (bX/bMag, bY/bMag)
    phi = np.arccos(aX_norm*bX_norm + aY_norm*bY_norm)
    
    if self.arc_orientation == 1:
      phi = 2*np.pi - phi
    else:
      pass
    
    self.linearExtent = self.radius*phi
  def getMotion(self, velocity, time):
    
    # Check if instructed motion is within the bounds of the object
    # This is done in linear image-space for ease of computation

    delta =  float(self.linearPos) + self.trans_dist2img_float(velocity*time)
    
    # if the instructed motion is within bounds, update linear position
    if delta < self.linearExtent and delta > 0:
      self.linearPos += self.trans_dist2img_float(velocity*time)
    
    # if the instructed motion is outside of the bounds, 
    if delta >= self.linearExtent:
      self.linearPos = 0
      self.realPos = self.start
      
      if self.objType == 'circle':
        self.theta = self.theta_start
      else:
        pass

      if self.nextObj is not None:
        runoff = delta - self.linearExtent
        # Because objects currently do not have an internal clock, distance can be passed in
        # as dist = vel, time = 1
        self.nextObj.getMotion(self.trans_img2dist_float(runoff), 1)
        print('traversing forward')
        return self.trans_img2dist_float(runoff)
        pass
      else:
        pass
    if (self.linearPos + self.trans_dist2img_float(velocity*time) < 0):
      print('traversing backwards is not yet supported ¯\_(ツ)_/¯')
      pass

    if (self.linearPos + self.trans_dist2img(velocity*time) < 0):
      print('traversing backwards is not yet supported ¯\_(ツ)_/¯')
      pass
    
    if self.objType == 'line':  
      (rX, rY) = self.realPos
      
      self.realPos = (rX + self.trans_dist2img_float(velocity*time*np.cos(self.angle)), 
                      rY + self.trans_dist2img_float(velocity*time*np.sin(self.angle)))
      
      (cX,cY) = (int(self.realPos[0]), int(self.realPos[1]))
      #self.realPos = self.trans_obj_2_img(cX,cY)
      
      cv2.circle(self.track_image, self.trans_obj_2_img(cX,cY), 5, self.color, -1)

    if self.objType == 'circle':      
      v = velocity*self.approach_direction

      delta_t = np.arctan(self.trans_dist2img(v*time)/self.radius)
      self.theta = self.theta + delta_t
      
      (radX, radY) = self.radius_point
      (posX, posY) = self.trans_obj_2_img(radX + self.radius*np.cos(self.theta), radY + self.radius*np.sin(self.theta))

      self.realPos = (int(posX), int(posY))
      cv2.circle(self.track_image, self.realPos, 5, self.color, -1)
    else:
      pass
  
  # Methods to draw object features on the track backround
  def draw_start_end(self): # marks object start end points in image space
    #get start and end points in image space
    (sX,sY) = self.trans_obj_2_img(self.start[0],self.start[1])
    (eX,eY) = self.trans_obj_2_img(self.end[0],self.end[1])

    #draw start and end points on image
    cv2.circle(self.track_image,(sX,sY), 7, self.color, -1)
    cv2.circle(self.track_image,(eX,eY), 7, self.color, -1)  
  def draw_radius_point(self):
    #same methodology as draw_start_end(), however will only draw if object is a cirlce
    if self.objType == 'circle':
      (cX, cY) = self.trans_obj_2_img(self.radius_point[0],self.radius_point[1])
      cv2.circle(self.track_image, (cX,cY), 5, self.color, -1)
    else:
      pass
  def draw_path(self): # Draws the path of the object from start point to end point
    """
    TODO:
      IS CURRENTLY: Draws out entire circle regardless of start/end points
      SHOULD BE: Should only draw arc between start and end

    """
    if self.objType == 'line': 
      #Line defined by start and end points measured in pixels, and transformed into object space
      cv2.line(self.track_image, self.trans_obj_2_img(self.start[0], self.start[1]), self.trans_obj_2_img(self.end[0], self.end[1]), self.color, 2)
    if self.objType == 'circle':
      #Circle defined by the given center point and derived radius value measured in pixels and transformed into object space
      (cX, cY) = self.trans_obj_2_img(self.radius_point[0],self.radius_point[1])
      cv2.circle(self.track_image, (cX,cY), self.radius, self.color, 2)
    else:
      pass

In [ ]:
# Build up The Polyline

#Read in and resize the image for the track backround
track = cv2.imread('/content/drive/MyDrive/(1) UCSD/(6) Fall 2022/MAE-148/AprilTags/track backround.png')
track = cv2.resize(track,(1920,1080), interpolation = cv2.INTER_AREA)

#Data to generate objects
"""
  start point         object type       color        radius point       circle parameter 1, circle parameter 2       
[((start_x, start_Y), 'line' or 'circle', (b, g, r), (center_x, center_y),       0 or 1,            0 or 1 or -1),
     end point
 ((end_x, end_y),      '*****',           (*, *, *), (***, ***),                 *,                 *)]

Becuase a polyline is continuous, the end of one object becomes the statr of the next object, so rather than duplicating this informaiton, 
the start of the next object is used as the end of the first, as shown here

Inputing circular objects requires information nessisary for correct path and motion interpretation.

circle parameter 1: If 0, the shorter distance between start and end points is assumed, 1 the longer path is chosen
circle parameter 2: If 1, motion is assumed to follow clockwise allong the path, -1 for counter-clockwise

"""
cps = [(-2.336, -1.133, 'line',   (0,255,0), (0,0),           0,  0),
       ( 1.754, -1.133, 'circle', (255,0,0), (1.754,0),       1,  1),
       ( 1.043,  0.882, 'line',   (0,0,255), (0,0),           0,  0),
       ( 0.182,  0.189, 'circle', (0,100,0), (-0.163,0.617),  0, -1),
       (-0.426,  0.134, 'line',   (100,0,0), (0,0),           0,  0),
       (-2.073,  1.031, 'circle', (0,0,100), (-2.336,0.548),  0,  1),
       (-2.885,  0.582, 'line',   (50,0,0),  (0,0),           0,  0),
       (-2.885, -0.582, 'circle', (0,50,0),  (-2.336,-0.582), 0,  1),
       (-2.336, -1.133, '',       (0,0,50),  (0,0),           0,  0)]

track_contour = polyLine(cps, track, 270)


Number of objects read in 8


Method to generate velocity inputs. 

In [ ]:
class velCurve():
  curveType =   None
  dist      =   None
  velMin    =   None
  velMax    =   None
  distance  =   None

  velocity  =   None
  position  =   None

  next      =   None

  def __init__(self, Velocity_curve_type, velocity_min, velocity_max, motion_distance):
    self.curveType = Velocity_curve_type
    self.velMin = velocity_min
    self.velMax = velocity_max
    self.distance = motion_distance    

    self.position = 0
    self.slope = 0

  def getVel(self, time):
    """
    TODO
    It would be nice to add some more velocity curve types

    """
    # Check if velocity is within its bounds
    if self.position < self.distance:
      if self.curveType == 'constant':
        # assign velocity to max
        self.velocity = self.velMax
        self.position += time*self.velocity
        #print(self.position)
        return [self.velMax, 0]
      if self.curveType == 'symmetric-ramp':
        # velocity ramps between min and max peaking in the middle
        if self.position == 0:
          self.velocity = self.velMin
          rise = self.velMax - self.velMin
          self.slope = (2*rise)/self.distance
        else:
          pass
        # ramp up on the first half 
        if self.position <= self.distance/2:
          self.velocity += self.slope*time
          self.position += time*self.velocity
          #print(self.velocity)
          #print(self.position)
        # ramp down in the middle half
        elif self.position > self.distance/2:
          self.velocity -= self.slope*time
          self.position += time*self.velocity
          #print(self.velocity)
          #print(self.position)
        else:
          # Poor man's exception handeling 
          print("something unexpected happened in getVel/ramp method")
      if self.curveType == 'sine':
        print("This feature has not been implemeted yet :/")
        pass        
    #If velocity is not within bounds of this object, pass to the next one
    else:
      try:
        self.position = 0
        return [next.velocity, 1] 
      except:
        print("Failed to hand over velocity to next object")
        return [self.velMin, 1]
      #Need to accomodate for handing off between velocity curves

In [ ]:
class tagReader():
  detector    =   None
  image       =   None
  results     =   None
  tagID       =   list()
  
  nodes = {1 : velCurve("constant", 0.5, 1, 5),
           2 : velCurve("constant", 0.5, 2, 5),
           3 : velCurve("constant", 0.5, 3, 5)}
  
  def __init__(self, detector, image):
    self.detector = detector
    self.image = image

  def setImage(self, img):
    self.image = img
  def readImage(self):
    #Getting the resutls from the detector, with the image input
    self.results = detector.detect(self.image)
    for r in self.results:
      self.tagID.append(int(r.tag_id))
    
    try:
      return self.nodes[self.tagID[len(tagID)-1]]
    except:
      print("Error detecting node")
      return None
  def annotateImage(self):
    img = cv2.cvtColor(self.image, cv2.COLOR_GRAY2RGB) # Converting the origonal image back to color, so that we can draw bounding boxes and stuff
    #loop to draw bounding boxes, and mark centers
    for r in result:
      # Setting corners
      (A,B,C,D) = r.corners 
      corner_B = (int(B[0]), int(B[1]))
      corner_C = (int(C[0]), int(C[1]))
      corner_D = (int(D[0]), int(D[1]))
      corner_A = (int(A[0]), int(A[1]))
      # Drawing lines between corners
      cv2.line(img, corner_A, corner_B, (0,255,0), 5)
      cv2.line(img, corner_B, corner_C, (0,255,0), 5)
      cv2.line(img, corner_C, corner_D, (0,255,0), 5)
      cv2.line(img, corner_D, corner_A, (0,255,0), 5)

      # Draw a circle in the center of each tag detected
      (cX,cY) = (int(r.center[0]), int(r.center[1]))
      cv2.circle(img,(cX,cY), 5, (0, 0, 255), -1)

      # Writing the detected tagIDs on the image
      tagFamily = r.tag_family.decode("utf-8")
      tagID = str(r.tag_id)
      print(tagID)
      cv2.putText(img, "tagID: " + tagID, (corner_A[0]+50, corner_A[1] - 30),
                  cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 10)
      
      print("[INFO] tag ID: {}".format(tagID))
      return(img)

In [ ]:
#Reading in the file, converting it to a black and white img, and resizing
img = cv2.imread('/content/drive/MyDrive/(1) UCSD/(6) Fall 2022/MAE-148/AprilTags/apriltag-imgs-master/apriltag-imgs-master/tag16h5/tag16_05_00003.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img,(4208,3120), interpolation = cv2.INTER_AREA)

apriltag.DetectorOptions() # Setting up the detector options object
options = apriltag.DetectorOptions(families="tag16h5",quad_decimate=1)
detector = apriltag.Detector(options) # Instantiating the detector, with the detector options

reader = tagReader(detector, img)

In [ ]:
curve = reader.readImage()

print(curve.getVel(1))


[3, 0]


In [ ]:
#track = cv2.imread('/content/drive/MyDrive/(1) UCSD/(6) Fall 2022/MAE-148/AprilTags/track backround.png')
#track = cv2.resize(track,(1920,1080), interpolation = cv2.INTER_AREA)

# vid = cv2.VideoWriter('/content/drive/MyDrive/(1) UCSD/(6) Fall 2022/MAE-148/AprilTags/output.avi',
#                       cv2.VideoWriter_fourcc('M','J','P','G'), 20, (1920,1080))

while False:
  track_contour.getMotion(1, 1/20)
# vid.write(track)
  time.sleep(0.25)    
  cv2_imshow(track_contour.track_image)

